In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


model_coverage = 15  # degrees
factor = 360/model_coverage

filename = "results/he/surface_flux_flibe_top.csv"
df = pd.read_csv(filename)

df

In [ ]:
# add another file to the datafram
filename = "results/he/surface_flux_inconel_inner_side.csv"

df2 = pd.read_csv(filename)

# add a new column to df
df["T flux surface 9"] = df2["T flux surface 9"]
df

In [ ]:
# add another file to the datafram
filename = "results/he/surface_flux_inconel_outer_side_calculated.csv"

df3 = pd.read_csv(filename)

# add a new column to df
df["T flux surface 9"] = df3["T flux surface 9"]
df

In [ ]:
# sum the duplicate times
df = df.groupby("t(s)").sum().reset_index()
df

In [ ]:
plt.plot(df["t(s)"], df["T flux surface 8"])

In [ ]:
import requests

# read experimental data
url = "https://raw.githubusercontent.com/LIBRA-project/BABY-1L-run-1/refs/tags/v0.5/data/processed_data.json"
experimental_data = requests.get(url).json()
cumulative_release_exp_inner = experimental_data["cumulative_tritium_release"]["IV"][
    "total"
]["value"]
sampling_times_inner = experimental_data["cumulative_tritium_release"]["IV"][
    "sampling_times"
]["value"]
cumulative_release_exp_outer = experimental_data["cumulative_tritium_release"]["OV"][
    "total"
]["value"]
sampling_times_outer = experimental_data["cumulative_tritium_release"]["OV"][
    "sampling_times"
]["value"]


In [ ]:
from scipy.integrate import cumulative_trapezoid
from libra_toolbox.tritium.model import quantity_to_activity, ureg

cumul_release = cumulative_trapezoid(df["T flux surface 8"], df["t(s)"], initial=0) * ureg.particle
cumul_release = quantity_to_activity(cumul_release).to(ureg.Bq)
cumul_release *= factor

s_to_days = 1 / 24 / 3600
plt.plot(df["t(s)"] * s_to_days, cumul_release)

plt.scatter(sampling_times_inner, cumulative_release_exp_inner * ureg.Bq)

In [ ]:
experimental_data

In [ ]:
TBR = experimental_data["measured_TBR"]["value"] * ureg.particle/ureg.neutron
salt_vol = 1 * ureg.L
neutron_rate = experimental_data["neutron_rate_used_in_model"]["value"] * ureg.neutron/ureg.s

tritium_source = (TBR/salt_vol*neutron_rate).to(ureg.particle/ureg.m**3/ureg.s)
print(f"{tritium_source:.2e~P}")

In [ ]:
from festim_model import irradiation_time, my_source


expected_plateau = irradiation_time * ureg.s * tritium_source * salt_vol.to(ureg.m**3)
expected_plateau = quantity_to_activity(expected_plateau).to(ureg.Bq)
expected_plateau